<a href="https://colab.research.google.com/github/joyalkenus/Project_imageSearch/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is the major run file of this project
- Aims to give a user friendly interface to interact with the system
- aims to integrate the BERT LLM to the RAM model to achieve proper image search for the user description.

In [1]:
#@title Import dependencies
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git clone https://github.com/joyalkenus/Project_imageSearch.git
%cd Project_imageSearch

Cloning into 'Project_imageSearch'...
remote: Enumerating objects: 743, done.
remote: Counting objects: 100% (743/743), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 743 (delta 387), reused 683 (delta 359), pack-reused 0
Receiving objects: 100% (743/743), 32.07 MiB | 16.74 MiB/s, done.
Resolving deltas: 100% (387/387), done.
/content/Project_imageSearch/Project_imageSearch


In [6]:
!pip install transformers

In [7]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap

clear_output()

In [9]:
# Download checkpoints
model_widget = widgets.Dropdown(
    options=["RAM", "Tag2Text"],
    value="RAM",
    description="Select mdoel:"
)
display(model_widget)

Dropdown(description='Select mdoel:', options=('RAM', 'Tag2Text'), value='RAM')

In [10]:
model = model_widget.value

In [11]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM
--2024-01-31 08:51:16--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 13.35.7.5, 13.35.7.38, 13.35.7.57, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.5|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-01-31 08:51:17--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_14m.pth%22%3

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model1 = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [22]:
def encode_texts(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        output = model1(**encoded_input)
    return output.last_hidden_state.mean(dim=1).numpy()




Use the Bert model to the capion data set



In [14]:
!python extract.py


/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l


In [15]:

df = pd.read_csv('image_tags.csv')

In [16]:
df['Tags']

0    condiment | counter top | table | dinning tabl...
1    baseball hat | boy | child | floor | girl | ma...
Name: Tags, dtype: object

In [17]:
#tags= df["Tags"]
#tags_split= [tags.strip() for tag in tags.split('|')]

In [18]:
# Create text input widget
text_input = widgets.Text(
    description='Enter Text:',
    placeholder='Type something'
)
# Create a button widget
button = widgets.Button(description="Submit")

In [19]:
# Output widget to display results
output = widgets.Output()

In [20]:
def on_button_clicked(b):
    global user_input
    user_input = text_input.value  # Assign the input text to user_input variable
    with output:
        print(f"You entered: {user_input}")

# Bind the button click event to the function
button.on_click(on_button_clicked)
# Display the widgets
display(text_input, button, output)

Text(value='', description='Enter Text:', placeholder='Type something')

Button(description='Submit', style=ButtonStyle())

Output()

In [30]:

def encode_text(text):
    # Your encoding logic here
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        output = model1(**encoded_input)
    text= output.last_hidden_state.mean(dim=1).numpy()
    return text

# Apply split on each element of the Series
df['Tags_split'] = df['Tags'].apply(lambda x: x.split(' | '))

# Processing each row in the DataFrame
for index, row in df.iterrows():
    tags_list = row['Tags_split']  # Now this is already a list of tags
    encoded_tags = encode_text(tags_list)  # Encode the list of tags
    print(tags_list)
    print(encoded_tags)  # Or use the encoded_tags as needed


['condiment', 'counter top', 'table', 'dinning table', 'plate', 'fill', 'food', 'fork', 'meal', 'noodle', 'peak', 'platter', 'salad', 'silverware', 'utensil', 'yellow']
[[-0.0682579   0.00166152 -0.11161387 ...  0.09540337 -0.4283763
  -0.11540041]
 [ 0.01749756  0.02653847 -0.348859   ...  0.08396428  0.02201242
  -0.07607406]
 [-0.23274902  0.20380485 -0.11796582 ... -0.01073592 -0.08852374
  -0.12971966]
 ...
 [ 0.21203291  0.34350067 -0.38957298 ...  0.0482294  -0.04708074
  -0.2921462 ]
 [-0.25496632  0.02687086 -0.53605056 ... -0.24391934 -0.13248616
   0.08757354]
 [-0.31074703  0.00624132 -0.28102988 ...  0.10842941  0.02707954
   0.29300982]]
['baseball hat', 'boy', 'child', 'floor', 'girl', 'man', 'pose', 'selfie', 'shirt', 'sit', 'smile', 'stool']
[[ 0.174272   -0.02464867 -0.58634925 ...  0.13525808 -0.2595209
  -0.30164802]
 [ 0.04219876 -0.00790249  0.04826237 ...  0.17808557 -0.02194765
  -0.20117304]
 [-0.03548129  0.04095047  0.03095319 ... -0.39099297 -0.07124539
  -0

In [28]:
user_embedding= encode_text(user_input)

In [31]:

# Assuming df['Tags'] contains the embeddings
embeddings = np.vstack(encoded_tags)
print(embeddings)

[[ 0.174272   -0.02464867 -0.58634925 ...  0.13525808 -0.2595209
  -0.30164802]
 [ 0.04219876 -0.00790249  0.04826237 ...  0.17808557 -0.02194765
  -0.20117304]
 [-0.03548129  0.04095047  0.03095319 ... -0.39099297 -0.07124539
  -0.20651126]
 ...
 [-0.13000533  0.13051383  0.12670761 ...  0.12903388 -0.18621042
  -0.06604587]
 [-0.03412683  0.10802042  0.06925949 ... -0.07121655 -0.17242315
  -0.03211841]
 [ 0.07539274  0.13871917 -0.14763165 ...  0.12517881  0.00665318
  -0.13865857]]


In [35]:
print("user_embedding shape:", np.array([user_embedding]).shape)
print("embeddings shape:", embeddings.shape)


user_embedding shape: (1, 1, 768)
embeddings shape: (12, 768)


In [36]:
# Reshape user_embedding to be two-dimensional
user_embedding = user_embedding.reshape(1, -1)

# Now user_embedding should have the shape (1, 768)
print("Reshaped user_embedding shape:", user_embedding.shape)


Reshaped user_embedding shape: (1, 768)


In [38]:


# Calculate cosine similarities
similarities = cosine_similarity(user_embedding, embeddings)

# Find the best match
best_match_index = similarities.argmax()
best_match_image_path = df.iloc[best_match_index]['image_path']

print(f"The most similar image is at: {best_match_image_path}")


The most similar image is at: /content/drive/My Drive/ImageSample/2.jpg
